Set the desired log level

In [ ]:
from sqil_core import logger
logger.setLevel(30)

## Pulsed experiments

#### Resonator spectroscopy

In [7]:
import numpy as np
# Convert amplitude to power dBm
10 + 20 * np.log10(0.03)

np.float64(-20.45757490560675)

In [ ]:
import numpy as np
from sqil_experiments.measurements.rr_spec import RRSpec, RRSpecOptions

rr_spec = RRSpec()
options = RRSpecOptions()
options.count = 2**10

rr_spec.qpu.quantum_elements[0].update(**{
    "readout_configuration": "transmission",
    "readout_amplitude": 0.1
})

result = rr_spec.run(
    [np.linspace(7e9, 8e9, 2001)],
    # [rr_spec.sweep_around("readout_resonator_frequency", 20e6, n_points=201)],
    # sweeps={"readout_amplitude": np.linspace(0.002, 0.03, 51)},
    qu_ids=["q0"],
    options=options,
)

#### Qubit specroscopy

In [ ]:
import numpy as np
from sqil_experiments.measurements.qu_spec import QuSpec, QuSpecOptions
from laboneq.dsl.enums import AveragingMode, AcquisitionType

qu_spec = QuSpec()
options = QuSpecOptions()
options.count = 2**12
options.averaging_mode = AveragingMode.CYCLIC  
options.acquisition_type = AcquisitionType.SPECTROSCOPY
# options.spectroscopy_reset_delay = 10e-6

transition = "ge"

qu_spec.qpu.quantum_elements[0].update(
    **{
        "current": 0.0e-3,
        "spectroscopy_amplitude": 0.8,
        "spectroscopy_length": 5e-6,
        "drive_lo_frequency": 4.6e9,
    }
)

result = qu_spec.run(
    [np.linspace(5.0e9, 6.0e9, 2001)],
    # [qu_spec.sweep_around(f"resonance_frequency_{transition}", 20e6, n_points=251)],
    transition=transition,
    qu_ids=["q0"],
    options=options,
    update_params=True,
    relevant_params=["readout_amplitude", "spectroscopy_amplitude"],
)

#### Time Rabi

In [ ]:
import numpy as np
from sqil_experiments.measurements.time_rabi import TimeRabi , TimeRabiOptions
from laboneq.dsl.enums import AcquisitionType, AveragingMode

time_rabi = TimeRabi()
options = TimeRabiOptions()
options.count = 2**12
options.acquisition_type = AcquisitionType.SPECTROSCOPY
options.averaging_mode = AveragingMode.CYCLIC

time_rabi.qpu.quantum_elements[0].update(
    **{
        "ge_drive_amplitude_pi": 0.8,
        # Make sure it's ALWAYS 1/2 of ge_drive_amplitude_pi
        "ge_drive_amplitude_pi2": 0.4,        
        "reset_delay_length": 60e-6,
    }
)

result = time_rabi.run(
    [np.linspace(20e-9, 600e-9, 53)],
    transition="ge",
    qu_ids=["q0"],
    options=options,
    update_params=True,
    relevant_params=["ge_drive_amplitude_pi", "resonance_frequency_ge"],
    # pulse_sheet=200e-9
)

#### Qubit temperature

In [ ]:
import numpy as np
from sqil_experiments.measurements.qubit_temperature import QubitTemperature, QubitTemperatureOptions
from laboneq.dsl.enums import AcquisitionType, AveragingMode

qubit_temp = QubitTemperature()
options = QubitTemperatureOptions()
options.count = 2**15
options.acquisition_type = AcquisitionType.SPECTROSCOPY

result = qubit_temp.run(
    [np.linspace(1e-6, 0.8, 2)],
    sweeps={"index": np.arange(3)},
    qu_ids=["q0"],
    options=options,
)

#### T1

In [ ]:
from sqil_experiments.measurements.T1 import T1
import numpy as np
from laboneq_applications.experiments.options import (
    TuneupExperimentOptions,
)
from laboneq.dsl.enums import AcquisitionType

T1_value = 50e-6
time = np.hstack(
    [
        np.linspace(0, T1_value, 11),
        np.logspace(np.log(T1_value * 1.1), np.log(5 * T1_value), 11, base=np.e),
    ]
)

options = TuneupExperimentOptions()
options.acquisition_type = AcquisitionType.SPECTROSCOPY
options.count = 2**12

T1_exp = T1()
result = T1_exp.run(
    [time],
    # sweeps={"index": np.arange(10)},
    options=options,
    transition="ge",
    update_params=False,
)

#### T2 echo

In [ ]:
from sqil_experiments.measurements.T2_echo import T2Echo, EchoExperimentOptions
import numpy as np
from laboneq.dsl.enums import AcquisitionType

T2_exp = T2Echo()

two_T1 = T2_exp.qubit_value("ge_T1", "q0") or 100e-6
time = np.hstack(
    [
        np.linspace(0, two_T1, 11),
        np.logspace(np.log(two_T1 * 1.1), np.log(5 * two_T1), 11, base=np.e),
    ]
)

options = EchoExperimentOptions()
options.acquisition_type = AcquisitionType.SPECTROSCOPY
options.count = 2**13

result = T2_exp.run(
    [time],
    # sweeps={"index": np.arange(10)},
    options=options,
    update_params=False
)

#### Interleaved T1-echo

In [ ]:
from sqil_experiments.measurements.interleaved_T1_echo import InterleavedT1Echo
from sqil_experiments.measurements.T2_echo import EchoExperimentOptions
import numpy as np
from laboneq.dsl.enums import AcquisitionType

interleaved_exp = InterleavedT1Echo()

T1_value = interleaved_exp.qubit_value("ge_T1", "q0")
time = np.hstack(
    [
        np.linspace(0, T1_value, 11),
        np.logspace(np.log(T1_value * 1.1), np.log(5 * T1_value), 11, base=np.e),
    ]
)

options = EchoExperimentOptions()
options.acquisition_type = AcquisitionType.SPECTROSCOPY
options.count = 2**12

result = interleaved_exp.run(
    [time],
    sweeps={"index": np.arange(200)},
    options=options,
    update_params=False
)

#### Ramsey

In [ ]:
from sqil_experiments.measurements.ramsey import Ramsey
import numpy as np
from laboneq_applications.experiments.options import (
    TuneupExperimentOptions,
)
from laboneq.dsl.enums import AcquisitionType

time = np.linspace(0.1e-6, 30e-6, 101)
detuning = 0.25e6

options = TuneupExperimentOptions()
options.acquisition_type = AcquisitionType.SPECTROSCOPY
options.count = 2**14

ramsey_exp = Ramsey()
result = ramsey_exp.run(
    [time],
    [detuning],
    sweeps={"index": np.arange(2)},
    options=options,
)

#### Dispersive shift

In [ ]:
import numpy as np
from sqil_experiments.measurements.dispersive_shift import DispersiveShift, DispersiveShiftOptions
import matplotlib.pyplot as plt

dispersive_shift = DispersiveShift()
options = DispersiveShiftOptions()
options.count = 2**11

result = dispersive_shift.run(
    [dispersive_shift.sweep_around("readout_resonator_frequency", 20e6, n_points=201)],
    qu_ids=["q0"],
    transition="ge",
    options=options,
)
plt.show()

#### IQ blobs

In [ ]:
import numpy as np
from sqil_experiments.measurements.iq_blobs import IQBlobs, IQBlobsOptions
from laboneq.dsl.enums import AveragingMode, AcquisitionType

single_shot = IQBlobs()
options = IQBlobsOptions()
options.count = 10000
options.acquisition_type = AcquisitionType.SPECTROSCOPY

single_shot.qpu.quantum_elements[0].update(
    **{
        # "current": 0.694e-3,
    }
)

result = single_shot.run(
    ['g', 'e', 'f'],
    qu_ids=["q0"],
    options=options,
    update_params=True,
)

#### T1 vs flux

In [ ]:
import numpy as np
from sqil_experiments.measurements.T1_adaptive import T1Adaptive
from sqil_experiments.measurements.qu_spec import QuSpecOptions
from sqil_experiments.measurements.time_rabi import TimeRabiOptions
from laboneq_applications.experiments.options import (
    TuneupExperimentOptions,
)
from laboneq.dsl.enums import AveragingMode, AcquisitionType

# Qubit spectroscopy options
spec_options = QuSpecOptions()
spec_options.count = 2**12
spec_options.averaging_mode = AveragingMode.CYCLIC  
spec_options.acquisition_type = AcquisitionType.SPECTROSCOPY
spec_params = [np.linspace(5.30e9, 5.40e9, 121)]

# Time Rabi options
rabi_options = TimeRabiOptions()
rabi_options.count = 2**11
rabi_options.acquisition_type = AcquisitionType.SPECTROSCOPY
rabi_options.averaging_mode = AveragingMode.CYCLIC
rabi_params = [np.linspace(20e-9, 600e-9, 53)]

# T1 options
T1_options =TuneupExperimentOptions()
T1_options.acquisition_type = AcquisitionType.SPECTROSCOPY
T1_options.count = 2**12
T1_params = None # Auto generated by experiment based on last T1

options = [spec_options, rabi_options, T1_options]
exp_params = [spec_params, rabi_params, T1_params]

T1_adaptive = T1Adaptive(no_instruments=True)
# General parameter updates (runs only once)
transition = "ge"
T1_adaptive = T1Adaptive(no_instruments=True)
T1_adaptive.qpu.quantum_elements[0].update(
    **{
        "spectroscopy_amplitude": 0.03,
        "spectroscopy_length": 5e-6, 
        "drive_lo_frequency": 4.8e9,

        "ge_drive_amplitude_pi": 0.2,
        "ge_drive_amplitude_pi2": 0.1,
        "reset_delay_length": 100e-6,

        "ge_T1": 50e-6
    }
)

T1_adaptive.run(
    exp_params,
    sweeps={"current": np.linspace(0.82e-3, 0.98e-3, 101)},
    options=options
)

#### T2 echo vs flux

In [ ]:
import numpy as np
from sqil_experiments.measurements.T2_echo_adaptive import T2EchoAdaptive
from sqil_experiments.measurements.qu_spec import QuSpecOptions
from sqil_experiments.measurements.time_rabi import TimeRabiOptions
from sqil_experiments.measurements.T2_echo import EchoExperimentOptions
from laboneq.dsl.enums import AveragingMode, AcquisitionType

# Qubit spectroscopy options
spec_options = QuSpecOptions()
spec_options.count = 2**12
spec_options.averaging_mode = AveragingMode.CYCLIC  
spec_options.acquisition_type = AcquisitionType.SPECTROSCOPY
spec_params = [np.linspace(5.30e9, 5.40e9, 121)]

# Time Rabi options
rabi_options = TimeRabiOptions()
rabi_options.count = 2**11
rabi_options.acquisition_type = AcquisitionType.SPECTROSCOPY
rabi_options.averaging_mode = AveragingMode.CYCLIC
rabi_params = [np.linspace(20e-9, 600e-9, 53)]

# T2 options
T2_options = EchoExperimentOptions()
T2_options.acquisition_type = AcquisitionType.SPECTROSCOPY
T2_options.count = 2**13
two_T1 = 28e-6
time = np.hstack(
    [
        np.linspace(0, two_T1, 11),
        np.logspace(np.log(two_T1 * 1.1), np.log(5 * two_T1), 11, base=np.e),
    ]
)
T2_params = [time]

options = [spec_options, rabi_options, T2_options]
exp_params = [spec_params, rabi_params, T2_params]

T2_adaptive = T2EchoAdaptive(no_instruments=True)
# General parameter updates (runs only once)
transition = "ge"
T2_adaptive = T2EchoAdaptive(no_instruments=True)
T2_adaptive.qpu.quantum_elements[0].update(
    **{
        "spectroscopy_amplitude": 0.03,
        "spectroscopy_length": 5e-6, 
        "drive_lo_frequency": 4.8e9,

        "ge_drive_amplitude_pi": 0.2,
        "ge_drive_amplitude_pi2": 0.1,
        "reset_delay_length": 125e-6,
    }
)

T2_adaptive.run(
    exp_params,
    sweeps={"current": np.linspace(0.1e-3, 1e-3, 301)},
    options=options
)

#### Qubit temperature vs flux

In [ ]:
import numpy as np
from sqil_experiments.measurements.qubit_temperature_adaptive import QubitTemperatureAdaptive
from sqil_experiments.measurements.qu_spec import QuSpecOptions
from sqil_experiments.measurements.time_rabi import TimeRabiOptions
from sqil_experiments.measurements.qubit_temperature import QubitTemperatureOptions
from laboneq_applications.experiments.options import (
    TuneupExperimentOptions,
)
from laboneq.dsl.enums import AveragingMode, AcquisitionType

# Qubit spectroscopy options
spec_ge_options = QuSpecOptions()
spec_ge_options.count = 2**13
spec_ge_options.averaging_mode = AveragingMode.CYCLIC  
spec_ge_options.acquisition_type = AcquisitionType.SPECTROSCOPY
spec_ge_params = [np.linspace(5.30e9, 5.40e9, 151)]
# ef
spec_ef_options = QuSpecOptions()
spec_ef_options.count = 2**14
spec_ef_options.averaging_mode = AveragingMode.CYCLIC  
spec_ef_options.acquisition_type = AcquisitionType.SPECTROSCOPY
spec_ef_params = [np.linspace(5.025e9, 5.175e9, 251)]

# Time Rabi options
rabi_ge_options = TimeRabiOptions()
rabi_ge_options.count = 2**11
rabi_ge_options.acquisition_type = AcquisitionType.SPECTROSCOPY
rabi_ge_options.averaging_mode = AveragingMode.CYCLIC
rabi_ge_params = [np.linspace(20e-9, 2400e-9, 53)]
# ef
rabi_ef_options = TimeRabiOptions()
rabi_ef_options.count = 2**13
rabi_ef_options.acquisition_type = AcquisitionType.SPECTROSCOPY
rabi_ef_options.averaging_mode = AveragingMode.CYCLIC
rabi_ef_params = [np.linspace(20e-9, 1600e-9, 53)]

# Qubit temperature options
qu_temp_options = QubitTemperatureOptions()
qu_temp_options.count = 2**15
qu_temp_options.acquisition_type = AcquisitionType.SPECTROSCOPY
qu_temp_params = [np.linspace(1e-6, 0.8, 2)]

options = [spec_ge_options, rabi_ge_options, spec_ef_options, rabi_ef_options, qu_temp_options]
exp_params = [spec_ge_params, rabi_ge_params, spec_ef_params, rabi_ef_params, qu_temp_params]

qt_adaptive = QubitTemperatureAdaptive(no_instruments=True)
# General parameter updates (runs only once)
qt_adaptive = QubitTemperatureAdaptive(no_instruments=True)
qt_adaptive.qpu.quantum_elements[0].update(
    **{
        "spectroscopy_amplitude": 0.8,
        "spectroscopy_length": 1e-6, 
        "drive_lo_frequency": 4.6e9,

        "ge_drive_amplitude_pi": 0.8,
        "ef_drive_amplitude_pi": 0.8,
        "reset_delay_length": 60e-6
    }
)

qt_adaptive.run(
    exp_params,
    sweeps={"current": np.linspace(0.1e-3, 0.91e-3, 300)},
    options=options
)
# 2.5 min => 8h = 200

## Continuous wave

#### Resonator spectroscopy

In [ ]:
import numpy as np
from sqil_experiments.measurements.cw_rr_spec import CW_RRSpec

rr_spec = CW_RRSpec()

rr_spec.qpu.quantum_elements[0].update(
    **{
        "readout_acquire_bandwith": 50,
        "readout_acquire_averages": 1
    }
)

result = rr_spec.run(
    [np.linspace(7e9, 8e9, 2001)],
    # [rr_spec.sweep_around("readout_resonator_frequency", 10e6, n_points=251)],
    # sweeps={"readout_power": np.linspace(-40, -20, 21)},
    qu_ids=["q0"],
    update_params=True,
)

#### Qubit spectroscopy

In [ ]:
import numpy as np
from sqil_experiments.measurements.cw_qu_spec import CW_QuSpec

qu_spec = CW_QuSpec()

fit_points = 1e-3 * np.array([-0.015, 0.020, 0.058, 0.0883, 0.1225, 0.140])

qu_spec.qpu.quantum_elements[0].update(
**{
    "current": 0.064e-3,
    "drive_power": -1,
    "readout_acquire_bandwith": 50,
    "readout_acquire_averages": 1
})

result = qu_spec.run(
    [np.linspace(8.1e9, 8.5e9, 251)],
    qu_ids=["q0"],
    update_params=False,
)